In [ ]:
'''
  Install packages required to connect to MySQL
  instance deployed in GCP. The connection is via
  Cloud SQL service offered by GCP
  NOTE: A session restart is required after installation
  of the cloud-sql-python-connector module
'''
# Install PyMySQL python module
!pip install cloud-sql-python-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.30.0 which is incompatible.


In [ ]:
'''
  Install the python MySQL module used for
  managing connections to MySQL DB from
  a python runtime
'''
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
'''
  Configure the environment with the GCP
  project to which the MySQL instance is
  assigned.
'''
from tools import load_db_cfg
dbcfg = load_db_cfg('cfg.json')
#project_id="youtubechannelanalysis-423709"
!gcloud config set project {dbcfg['project_id']}

Updated property [core/project].


In [ ]:
'''
  Aunthenticate to Google Colab as a valid
  GCP user that has access to the above
  project and its resources.
  NOTE: The project needs to be pre-configured
  for the user access
'''
from google.colab import auth
auth.authenticate_user()


In [ ]:
import pandas as pd
from ytdbconnect import  YtDbConnector
import sqlalchemy

'''
  Data Access Object (DAO) that could be used to store and load
  channel, playlist, comments & vieos
'''
class YtDao:
  sql = {
      'add_channel':'''INSERT INTO channel (id,name,type,views,description,status,playlist_id) VALUES (:id,:name,:type,:views,:description,:status,:playlist_id);''',
      'get_channel':'''SELECT c.id 'Channel Id',c.name 'Channel Name',c.type 'Channel Type',c.views 'View Count',c.description 'Description',c.status 'Status',c.playlist_id 'Playlist Id', count(*) 'Video Count' FROM channel c JOIN video v ON (c.id=v.channel_id) WHERE c.id=%(id)s;''',
      'get_supported_channels':'''SELECT id,name,description FROM channel ORDER BY name;''',
      'add_playlist':'''INSERT INTO playlist (id,channel_id,name) VALUES (:id,:channel_id,:name);''',
      'get_playlists_in_channel': '''SELECT id 'Playlist Id',name 'Playlist Name',channel_id 'Channel Id' FROM playlist WHERE channel_id=%(channel_id)s;''',
      'add_comment':'''INSERT INTO comment (id,video_id,author,published_date,comment_text) VALUES (:id,:video_id,:author,STR_TO_DATE(:published_date,"%Y-%m-%dT%H:%i:%sZ"),:comment_text);''',
      'get_comments_in_channel':'''SELECT id 'Comment Id',video_id 'Video Id',comment_text 'Comment Text',author Author,published_date 'Published Date' FROM comment WHERE video_id IN (SELECT id FROM video WHERE channel_id=%(channel_id)s);''',
      'add_video':'''INSERT INTO video (id,playlist_id,channel_id,name,description,published_date,view_count,like_count,dislike_count,favorite_count,comment_count,duration,caption_status) VALUES (:id,:playlist_id,:channel_id,:name,:description,STR_TO_DATE(:published_date,"%Y-%m-%dT%H:%i:%sZ"),:view_count,:like_count,:dislike_count,:favorite_count,:comment_count,:duration,:caption_status);''',
      'get_videos_in_channel':'''SELECT id 'Video Id',playlist_id 'Playlist Id',channel_id 'Channel Id',name 'Video Name',description 'Video Description', published_date 'Published Date',view_count 'View Count',like_count 'Like Count',dislike_count 'Dislike Count',favorite_count 'Favourite Count',comment_count 'Comment Count',duration 'Duration (secs)',caption_status 'Caption Status' FROM video WHERE video.playlist_id IN (SELECT playlist_id FROM channel WHERE channel_id=%(channel_id)s);''',
      'add_thumbnail':'''INSERT INTO thumbnails_in_video (video_id,type,url) VALUES (":video_id",":type",":url");''',
      'get_thumbnails_in_video': '''SELECT video_id,type,url,video_id FROM thumbnails_in_video WHERE video_id IN (SELECT id from video WHERE channel_id=%(channel_id)s));''',

  }

  __supported_stats__ = {
      'S1':{'description':'What are the names of all the videos and their corresponding channels?',
            'sql':'''SELECT v.name 'Video Name',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id);'''},
      'S2':{'description':'Which channels have the most number of videos, and how many videos do they have?',
            'sql':'''SELECT c.id 'Channel Id',c.name 'Channel Name',count(v.id) 'VideoCount' FROM channel c JOIN video v ON (c.id=v.channel_id) GROUP BY c.id,c.name ORDER BY VideoCount DESC'''},
      'S3':{'description':'What are the top 10 most viewed videos and their respective channels?',
            'sql':'''SELECT v.id 'Video Id',v.name 'Video Name',v.view_count 'View Count',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id) ORDER BY view_count DESC LIMIT 10'''},
      'S4':{'description':'How many comments were made on each video, and what are their corresponding video names?',
            'sql':'''SELECT id 'Video Id', name 'Video Name', comment_count 'Comment Count' FROM video ORDER BY comment_count DESC'''},
      'S5':{'description':'Which videos have the highest number of likes, and what are their corresponding channel names?',
            'sql':'''SELECT v.id 'Video Id',v.name 'Video Name', v.like_count 'Likes Count',c.name 'Channel Name' FROM video v JOIN channel c ON (v.channel_id=c.id) ORDER BY v.like_count DESC LIMIT 50;'''},
      'S6':{'description':'What is the total number of likes and dislikes for each video, and what are their corresponding video names?',
            'sql':'''SELECT id 'Video Id', name 'Video Name', like_count 'Likes Count', dislike_count 'Dislikes Count' FROM video ORDER BY like_count DESC'''},
      'S7':{'description':'What is the total number of views for each channel, and what are their corresponding channel names?',
            'sql':'''SELECT id 'Channel Id',name 'Channel Name',views 'Channel Views' FROM channel ORDER BY views DESC'''},
      'S8':{'description':'What are the names of all the channels that have published videos in the year 2022',
            'sql':'''SELECT c.id 'Channel Id',c.name 'Channel Name' FROM channel c WHERE c.id IN (SELECT video.channel_id FROM video WHERE YEAR(published_date)=2022);'''},
      'S9':{'description':'What is the average duration of all videos in each channel, and what are their corresponding channel names?',
            'sql':'''SELECT v.channel_id 'Channel Id',c.name 'Channel Name',AVG(v.duration) 'Avg. Duration (secs)' FROM video v JOIN channel c ON (v.channel_id=c.id) GROUP BY v.channel_id, c.name ORDER BY AVG(v.duration) DESC;'''},
      'S10':{'description':'Which videos have the highest number of comments, and what are their corresponding channel names?',
             'sql':'''SELECT v.id 'Video Id',v.name 'Video Name',c.name 'Channel Name', v.comment_count 'Comment Count' FROM video v JOIN channel c ON(v.channel_id=c.id) ORDER BY comment_count DESC'''}
  }

  def __init__(self,dbcfg):
    self.__ytd_connector__ = YtDbConnector(dbcfg)
    self.__connection__ = self.__ytd_connector__.connect()

  def is_connected(self):
    if self.__connection__ is None:
      return False
    else:
      return True

  '''
    Get all the statistics that are supported by this DAO
  '''
  def get_supported_stats(self):
    ret = dict()
    df = pd.DataFrame(self.__supported_stats__)
    return df.T['description']
    #for key in self.__supported_stats__.keys():
     # ret[key]=self.__supported_stats__[key]['description']
    #return ret


  def get_stat(self, stat_key):
    df=pd.read_sql_query(self.__supported_stats__[stat_key]['sql'], self.__connection__)
    return df


  '''
    Insert channel information and retun the inserted information
    as a dataframe
  '''
  def __add_channel__(self,channel):
    stmt = sqlalchemy.text(self.sql['add_channel'],)
    self.__connection__.execute(stmt,{'id': channel['id'], 'name': channel['name'], 'type': channel['type'], 'views': channel['views'], 'description': channel['description'], 'status': channel['status'],'playlist_id':channel['playlistId']})

  def add_channel(self,channel):
      self.__add_channel__(channel)
      self.__connection__.commit()
      df=self.get_channel(channel['id'])
      return df


  '''
    Retrieve channel information for the specified channel and
    return it as a dataframe
  '''
  def get_channel(self, channel_id):
      df=pd.read_sql_query(self.sql['get_channel'], self.__connection__,params={'id':channel_id})
      return df

  '''
    Retrieve the list of channels that are already in the db
  '''
  def get_supported_channels(self):
      df=pd.read_sql_query(self.sql['get_supported_channels'], self.__connection__)
      return df

  '''
    Insert playlists for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_playlists__(self,channel_id,playlists):
    stmt = sqlalchemy.text(self.sql['add_playlist'],)
    for key in playlists.keys():
      playlist=playlists[key]
      self.__connection__.execute(stmt,{'id':key, 'name': playlist['name'], 'channel_id': channel_id})

  def add_playlists(self,channel_id, playlists):
    self.__add_playlists__(channel_id,playlists)
    self.__connection__.commit()
    df = self.get_playlists_by_channel(channel_id)
    return df

  '''
    Retrieve all playlists for the specified channel and
    return them in dataframe
  '''
  def get_playlists_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_playlists_in_channel'], self.__connection__, params={'channel_id':channel_id})

  '''
    Insert comment threads for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_comments__(self,channel_id, comments):
    stmt = sqlalchemy.text(self.sql['add_comment'],)
    for key in comments.keys():
      comment=comments[key]
      self.__connection__.execute(stmt,{'id': comment['id'],'video_id': comment['video_id'],'author': comment['author'],'published_date': comment['published_date'],'comment_text':comment['text']})

  def add_comments(self,channel_id, comments):
    self.__add_comments__(channel_id, comments)
    self.__connection__.commit()
    df=self.get_comments_by_channel(channel_id)
    return df

  '''
    Retrieve all comment threds for the specified channel and
    return them in dataframe
  '''
  def get_comments_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_comments_in_channel'], self.__connection__, params={'channel_id':channel_id})

  '''
    Insert video info for the specified channel and retun inserted info in
    a dataframe
  '''
  def __add_videos__(self,channel_id, videos):
    stmt = sqlalchemy.text(self.sql['add_video'],)
    for key in videos.keys():
      video=videos[key]
      self.__connection__.execute(stmt,{'id':video['id'],'playlist_id':video['playlist_id'],'channel_id':video['channel_id'],'name':video['name'],'description':video['description'],'published_date':video['published_date'],'view_count':video['view_count'],'like_count':video['like_count'],'dislike_count':video['dislike_count'],'favorite_count':video['favorite_count'],'comment_count':video['comment_count'],'duration':video['duration'],'caption_status':video['caption_status']})

  def add_videos(self,channel_id, videos):
    self.__add_videos__(channel_id, videos)
    self.__connection__.commit()
    df=self.get_videos_by_channel(channel_id)
    return df

  '''
    Retrieve all video for the specified channel and
    return them in dataframe
  '''
  def get_videos_by_channel(self, channel_id):
    return pd.read_sql_query(self.sql['get_videos_in_channel'], self.__connection__, params={'channel_id': channel_id},index_col="id")

  def __add_thumbnails__(self,videos):
    stmt = sqlalchemy.text(self.sql['add_thumbnail'],)
    for video in videos.values():
      for key in video['thumbnails'].keys():
        url= video['thumbnails'][key]
        self.__connection__.execute(stmt,{'video_id': video['id'],'type': key,'url': url})

  def get_thumbnails_in_channel(self,channel_id):
    return pd.read_sql_query(self.sql['get_thumbnails_in_channel'], self.__connection__, params={'channel_id': channel_id},index_col="video_id")

  def close(self):
    self.__connection__.close()


  '''
    Add all the channel information in oneshot
  '''
  def add_channel_info(self, channel, playlists, comments, videos):
    self.__add_channel__(channel)
    self.__add_playlists__(channel['id'],playlists)
    self.__add_videos__(channel['id'], videos)
    self.__add_comments__(channel['id'], comments)
    #self.__add_thumbnails__(videos)
    self.__connection__.commit()







In [ ]:
import traceback
from tools import load_db_cfg
'''
  Test program for retrieving the stats from DB
'''
try:
  dbcfg= load_db_cfg('cfg.json')
  dao = YtDao(dbcfg)
  stat_key='S10'
  df_ss= dao.get_supported_stats()
  print(f'''\n{dao.get_stat(stat_key).to_markdown()}''')
  print(f'''\n Supported Stats: {dao.get_supported_stats().to_markdown()}''')
  print(f'''\n Supported Channels:\n {dao.get_supported_channels().to_markdown()}''')
except:
  print(f'''Failed to get stats for key {stat_key}''')
  print(traceback.print_exc())

print('\nConnection closed .. !')
dao.close()

Successfully connected to 'youtube' database!! 

|     | Video Id    | Video Name                                                                                             | Channel Name         |   Comment Count |
|----:|:------------|:-------------------------------------------------------------------------------------------------------|:---------------------|----------------:|
|   0 | da1vvigy5tQ | Reversing Type 2 diabetes starts with ignoring the guidelines | Sarah Hallberg | TEDxPurdueU           | TEDx Talks           |           20488 |
|   1 | tPEU-ze_C54 | 'You're lying': George Conway clashes with Republican commentator over Trump guilty verdict            | CNN                  |           18305 |
|   2 | Ja168gMpb3o | 3 - Why This Kolaveri Di Video | Dhanush, Shruti | Anirudh                                             | SonyMusicSouthVEVO   |           10686 |
|   3 | GdN9qdDnE5U | Aarya-2 - My Love Is Gone Video | Allu Arjun | Devi Sri Prasad                           

In [ ]:
import traceback
from tools import load_db_cfg

test_channels={
    'Tamil Pokkisham': 'UCS84kz7Fs8bzRs6xcPY9lQQ',
    'CNN': 'UCupvZG-5ko_eiXAupbDfxWw',
    "TEDx Talks":'UCsT0YIqwnpJCM-mx7-gSA4Q',
    "WION":"UC_gUM8rL-Lrg6O3adPW9K1g",
    "The Indian Mystics":"UCcnBJHMugWpuy6Y7-SlI1ew",
    "Future Technology":"UCoIPNbr4UXhhZXDBZaUNHwA",
    "SonyMusicSouthVEVO":"UCTNtRdBAiZtHP9w7JinzfUg",
    "NBA":"UCWJ2lWNubArHWmf3FIHbfcQ",
    "moneycontrol":"UChftTVI0QJmyXkajQYt2tiQ",
    "GUVI":"UCduIoIMfD8tT3KoU0-zBRgQ",
    "Neural networks":"UCYO_jab_esuFRV4b17AJtAw"
}
dao=YtDao(load_db_cfg('cfg.json'))
print(dao.get_channel('UCHU81qoYaZinuhdLoruBQ9g').to_markdown())
dao.close()
print('Connection closed .. !')


Successfully connected to 'youtube' database!! 
|    | Channel Id               | Channel Name   | Channel Type    |   View Count | Description                                                                                                                                                                                                                                                                                                                | Status   | Playlist Id              |   Video Count |
|---:|:-------------------------|:---------------|:----------------|-------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------|:-------------------------|--------------:|
|  0 | UCHU81qoYaZinuhdLoruBQ9g | Simply C

In [ ]:
'''
  YtDao Tests verifying individual table commits
'''
import traceback
from tools import load_db_cfg
test_channels={
    'Tamil Pokkisham': 'UCS84kz7Fs8bzRs6xcPY9lQQ',
    'CNN': 'UCupvZG-5ko_eiXAupbDfxWw',
    "TEDx Talks":'UCsT0YIqwnpJCM-mx7-gSA4Q',
    "WION":"UC_gUM8rL-Lrg6O3adPW9K1g",
    "The Indian Mystics":"UCcnBJHMugWpuy6Y7-SlI1ew",
    "Future Technology":"UCoIPNbr4UXhhZXDBZaUNHwA",
    "SonyMusicSouthVEVO":"UCTNtRdBAiZtHP9w7JinzfUg",
    "NBA":"UCWJ2lWNubArHWmf3FIHbfcQ",
    "moneycontrol":"UChftTVI0QJmyXkajQYt2tiQ",
    "GUVI":"UCduIoIMfD8tT3KoU0-zBRgQ",
    "Neural networks":"UCYO_jab_esuFRV4b17AJtAw"
}

# Mock Data
channel_id=''
channel=playlists=videos=comments={}
# End Mock Data
try:
  dao = YtDao(load_db_cfg('cfg.json'))
  try:
    print(f'''\n{dao.add_channel(channel).to_markdown()}''')
    try:
      print(f'''\n{dao.add_playlists(channel_id,playlists).to_markdown()}''')
      try:
          print(f'''\n{dao.add_videos(channel_id,videos).to_markdown()}''')
          try:
            print(f'''\n{dao.add_comments(channel_id,comments).to_markdown()}''')
          except:
            print('add_comments() failed.')
            print(traceback.print_exc())
      except:
          print('add_videos() failed.')
          print(traceback.print_exc())
    except:
      print('add_playlists() failed.')
      print(traceback.print_exc())
  except:
    print('add_channel() failed')
    print(traceback.print_exc())

except:
  print('Connector init failed.')
  print(traceback.print_exc())



print('Connection closed .. !')
dao.close()


In [ ]:
'''
  YtDao Tests verifying oneshot commit
'''
import traceback
test_channels={
    'Tamil Pokkisham': 'UCS84kz7Fs8bzRs6xcPY9lQQ',
    'CNN': 'UCupvZG-5ko_eiXAupbDfxWw',
    "TEDx Talks":'UCsT0YIqwnpJCM-mx7-gSA4Q',
    "WION":"UC_gUM8rL-Lrg6O3adPW9K1g",
    "The Indian Mystics":"UCcnBJHMugWpuy6Y7-SlI1ew",
    "Future Technology":"UCoIPNbr4UXhhZXDBZaUNHwA",
    "SonyMusicSouthVEVO":"UCTNtRdBAiZtHP9w7JinzfUg",
    "NBA":"UCWJ2lWNubArHWmf3FIHbfcQ",
    "moneycontrol":"UChftTVI0QJmyXkajQYt2tiQ",
    "GUVI":"UCduIoIMfD8tT3KoU0-zBRgQ",
    "Neural networks":"UCYO_jab_esuFRV4b17AJtAw"
}

# Mock data
channel=playlists=comments=videos=dict()
# Insert mockdata here..
# End Mock
from tools import load_db_cfg
dao = YtDao(load_db_cfg('cfg.json'))
dao.add_channel_info(channel,playlists,comments,videos)
dao.close()